# Important Libraries

In [2]:
from bs4 import BeautifulSoup
import requests

# test

In [3]:
url = "https://www.everydayhealth.com/abdominal-pain/guide/#q-as"

In [4]:
response = requests.get(url)
if response.status_code == 200:
    HTML = BeautifulSoup(response.text, 'html.parser')
else:
    raise Exception("Status code {}".format(response.status_code))

In [7]:
html1 =HTML.find("article",class_="eh-template eh-template--article eh-template--rails eh-template--right-rail eh-template--left-rail")
print(html1 is not None)

True


In [14]:
(html1.find("div",class_="eh-template__top").find("div",class_="subheader-label eh-pg-headline__subheader").text,
 html1.find("div",class_="eh-template__top").find("h1",class_="eh-pg-headline__title").text)

('Abdominal Pain',
 'What Is Abdominal Pain? Symptoms, Causes, Diagnosis, Treatment, and Prevention')

In [18]:
html2 = html1.find("div",class_="eh-template__body")
print(html2 is not None)

True


In [21]:
contexts = html2.findAll("div",class_="eh-widget eh-widget--cb")
print(bool(contexts))

True


In [23]:
contexts [0].text

'Most people experience abdominal discomfort or pain at some point in their lives. Abdominal pain (sometimes called stomachache or bellyache) is usually felt in the part of the trunk below the ribs, above the pelvis and the groin. It can range in intensity from a mild ache to severe, disabling pain.While abdominal pain isn’t normal, it isn’t necessarily serious, and it often resolves itself.But certain forms of abdominal pain may indicate a serious health condition, so it’s important to recognize the signs that may indicate you have an underlying problem requiring medical attention.[1]'

In [28]:
[ link.get("href") for link in contexts [1].findAll("a") if link.get("href").startswith("https://www.everydayhealth.com") ]

['https://www.everydayhealth.com/appendicitis/guide/appendix/']

In [42]:
import json

In [33]:
import re

In [ ]:
re.sub

In [39]:
re.sub("[^a-zA-Z/d ]","","heloo, mrs.")

'heloo mrs'

In [50]:
import time

In [54]:
skip_urls = []
fetched_urls = []
def get_documents(urls:list,skipped_url = skip_urls,fetched_urls=fetched_urls)->None:
    
    links = []
    for url in urls:
        if url in fetched_urls:
            continue
        
        else:
            fetched_urls.append(url)
        
        time.sleep(1)
        response = requests.get(url)
        
        if response.status_code == 200:
            HTML = BeautifulSoup(response.text, 'html.parser')
        else:
            raise Exception("Status code {}".format(response.status_code))
        
        html1 =HTML.find("article",class_="eh-template eh-template--article eh-template--rails eh-template--right-rail eh-template--left-rail")
        
        if html1 is None:
            skip_urls.append(url)
            continue
        
        try:
            category_ = html1.find("div",class_="eh-template__top").find("div",class_="subheader-label eh-pg-headline__subheader").text
        except:
            category_ = ""
        try:
            head = html1.find("div",class_="eh-template__top").find("h1",class_="eh-pg-headline__title").text
        except:
            continue
        
        html2 = html1.find("div",class_="eh-template__body")
        if html2 is None:
            skip_urls.append(url)
            continue
            
        contexts = html2.findAll("div",class_="eh-widget eh-widget--cb")
        if not contexts:
            skip_urls.append(url)
            continue
        
        documents = []
        
        for context in contexts:
            documents.append(
                {
                    "url":url,
                    "category":category_,
                    "head":head,
                    "context":context.text
                }
            )
            
            links.extend([ link.get("href") for link in context.findAll("a") if link.get("href").startswith("https://www.everydayhealth.com") ])
            
        
        
        with open(f'../data/{re.sub("[^a-zA-Z/d ]","",head).replace(" ","_")}.json',"w") as f:
            json.dump(documents,f)
        
        del documents,html2,html1
        
    if links:
        skipped_url.extend( get_documents(links,skip_urls,fetched_urls) )
        
        return skip_urls
            

        

In [55]:
get_documents(['https://www.everydayhealth.com/appendicitis/guide/appendix/'])

KeyboardInterrupt: 

In [57]:
len(skip_urls)

590

In [59]:
import os
documents = os.listdir('../data')

In [60]:
len(documents)

608

In [61]:
skip_urls

['https://www.everydayhealth.com/digestive-health/common-digestive-conditions-from-top-bottom/',
 'https://www.everydayhealth.com/digestive-health-pictures/icky-but-interesting-facts-about-poop.aspx',
 'https://www.everydayhealth.com/copd/',
 'https://www.everydayhealth.com/drugs/aspirin',
 'https://www.everydayhealth.com/drugs/ibuprofen',
 'https://www.everydayhealth.com/drugs/naproxen',
 'https://www.everydayhealth.com/biopsy/guide/',
 'https://www.everydayhealth.com/proton-pump-inhibitors/guide/',
 'https://www.everydayhealth.com/drugs/nexium',
 'https://www.everydayhealth.com/drugs/prevacid',
 'https://www.everydayhealth.com/drugs/prilosec',
 'https://www.everydayhealth.com/drugs/pantoprazole',
 'https://www.everydayhealth.com/drugs/aciphex',
 'https://www.everydayhealth.com/drugs/cimetidine',
 'https://www.everydayhealth.com/drugs/pepcid',
 'https://www.everydayhealth.com/drugs/nizatidine',
 'https://www.everydayhealth.com/blood-transfusion/guide/',
 'https://www.everydayhealth.co

In [62]:
len(fetched_urls)

1217

In [63]:
from langchain_community.document_loaders import JSONLoader


In [83]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["source"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["head"] = record.get("head")

    return metadata



loader = JSONLoader(
    file_path='../data/_Steps_to_Help_Prevent_Sepsis_if_You_Have_an_Infection.json',
    jq_schema='.[]',
    content_key="context",
    metadata_func=metadata_func
)

data = loader.load()

In [84]:
data[0]

Document(metadata={'source': 'https://www.everydayhealth.com/sepsis/preventing-sepsis-after-infection-key-steps-take/', 'seq_num': 1, 'category': 'Sepsis', 'head': '5 Steps to Help Prevent Sepsis if You Have an Infection'}, page_content='Sepsis is a life-threatening complication that occurs when chemicals released into the bloodstream to stop an infection cause widespread inflammation. (1)Symptoms of sepsis include a fever, a fast heart rate, and a fast respiratory rate. Sepsis can progress to severe sepsis, at which point you may have dizziness, confusion, and difficulty breathing. If sepsis is left untreated, the next stage is septic shock.“Shock is a state where organs don’t receive the appropriate blood flow to function due to severely low blood pressure,” says\xa0Kimberly Brown, MD, MPH, an ER doctor in Memphis, Tennessee.Sepsis can occur after an infection, whether it’s bacterial, viral, or fungal. Common infections that can turn into sepsis include urinary tract infections, pneu

In [85]:
print(data[4].page_content)

4. Wash Your Hands and Practice Good HygieneAnother way to prevent infections and ultimately sepsis is to practice good hygiene. This involves frequently washing your hands to reduce the spread of bacteria, viruses, and fungi, explains Rebecca Park, RN, who is based in New York City and the founder of the natural health resource Remedies For Me.Washing entails more than running water over your hands. You should use warm or hot soapy water, and lather your hands. To thoroughly remove germs and bacteria, rub your hands together vigorously, making sure that the water cleans in between your fingers and underneath nails. (4)As a general rule of thumb, it should take about 15 to 20 seconds to wash your hands the right way. You should wash your hands for as long as it takes to sing the “Happy Birthday” song twice. When you’re finished, rinse and dry with a clean cloth. (4)It’s important that you also teach your children proper hand washing.Along with knowing the right way to wash your hands, 

In [86]:
data[4].metadata

{'source': 'https://www.everydayhealth.com/sepsis/preventing-sepsis-after-infection-key-steps-take/',
 'seq_num': 5,
 'category': 'Sepsis',
 'head': '5 Steps to Help Prevent Sepsis if You Have an Infection'}

In [88]:
from pathlib import Path

In [90]:
# Define the documents extraction function.

def doc_loder(file_path)->list:
    loader = JSONLoader(
        file_path=file_path,
        jq_schema='.[]',
        content_key="context",
        metadata_func=metadata_func
    )
    
    return loader.load()

In [94]:
directory = Path('../data')
files = os.listdir(directory)
documents = []
for file in files:
    file_path = directory / file
    documents.extend(doc_loder(file_path))

In [95]:
len(documents)

5129

In [97]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [99]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=documents,
    embedding=embedding_function,
    persist_directory="../chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [106]:
vectorstore1 = Chroma.load("../chroma_db")

AttributeError: type object 'Chroma' has no attribute 'load'

In [102]:
query = "What Is Psychedelic TherapyWhat Is Psychedelic Therapy?"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'What Is Psychedelic TherapyWhat Is Psychedelic Therapy?'

Result 1:
Source: https://www.everydayhealth.com/integrative-health/potential-health-benefits-of-microdosing-psychedelics/
Content: What Is Psychedelic Therapy?Psychedelic therapy involves ingesting psychedelic substances, such as psilocybin (from “magic mushrooms”), LSD, MDMA, or ketamine, to experience an altered consciousness and potentially cause changes in the brain. The theory is that these changes can alleviate mental health conditions, such as PTSD or depression.[1]But there are several downsides to psychedelic therapy, including legal complexities, potential side effects, and adverse psychological challenges. Treatment is also quite intensive, often requiring multiple lengthy sessions before there's a noticeable effect.Microdoses of psychedlics may counter some of these downsides.Please note: This article serves only as an informational resource about what we know on microdosi